In [ ]:
from extended_source_files import *
from plot_helpers import *
from raster_compare.plots import PlotBase

import query_helpers

import math
import pandas as pd

# Analysis

In [ ]:
mask_to_snow_depth()

df = pd.DataFrame({
        'aso_snow_depth': aso_snow_depth_values.ravel(),
        'sfm_snow_depth': sfm_snow_depth_values.ravel(),
        'sd_difference': sd_difference_values.ravel(),
        'elevation': dem_values.ravel(),
        'slope': dem.slope.ravel(),
        'aspect': dem.aspect.ravel(),
        'hillshade_snow_on': hillshade_snow_on.ravel(),
        'hillshade_snow_free': hillshade_snow_free.ravel(),
        'casi_class': casi_classification.ravel(),
})

In [ ]:
# TODO - Only drop rows that have no ASO and SfM NaN
df.dropna(inplace=True)
df['elevation'] = df['elevation'].astype('int16')
df['slope'] = df['slope'].astype('int16')
df['aspect'] = df['aspect'].astype('int16')
df['hillshade_snow_on'] = df['hillshade_snow_on'].astype('int16')
df['hillshade_snow_free'] = df['hillshade_snow_free'].astype('int16')
df['casi_class'] = pd.cut(df['casi_class'], CASI_MAPPING, labels=CASI_CLASSES)

positive_sfm = query_helpers.get_positive(df, 'sfm_snow_depth')
negative_sfm = query_helpers.get_negative(df, 'sfm_snow_depth')

## Snow depth by elevation (positive SfM values)

**Snow Depth binning**
* 0.1 m from 0 to 15 m
* 0.5 m from 15 m to 50 m

In [ ]:
max_sd = math.ceil(positive_sfm.sfm_snow_depth.max())
bins = np.concatenate((
    np.arange(0, 15.10, 0.10),
    np.arange(15, max_sd, 0.25),
))

elevation_bands = 10
elevation_min = df.elevation.min() - df.elevation.min() % elevation_bands
elevation_max = df.elevation.max() + (elevation_bands - df.elevation.max() % elevation_bands)
elevation_range = np.arange(elevation_min, elevation_max + elevation_bands, elevation_bands)

hist_opts = dict(
    bins=[bins, elevation_range],
    vmin=0,
    vmax=150,
    cmin=1,
)

COLOR_BAR_ATTR = dict(right=0.9, rect=[0.91, 0.125, 0.03, 0.795])
FIGURE_SPECS = dict(figsize=(14,7), dpi=150)
HIST2D_PLOT = dict(ncols=2, sharey=True, sharex=True, **FIGURE_SPECS)

def plot_hist2d(ax, x_data, y_data, title, **kwargs):
    ax.set_facecolor('whitesmoke')
    if len(title) > 0:
        ax.set_xlabel(SNOW_DEPTH_LABEL)
        ax.set_title(title)
    ax.set_xlim(left=-.2)
    ax.tick_params(axis='x', direction='inout', length=10)
    ax.tick_params(axis='y', length=6)
    ax.xaxis.set_ticks_position('both')

    return ax.hist2d(x_data, y_data, **kwargs)

def scatter_inset(ax, data, label):
    bin_count_sum = np.count_nonzero(~np.isnan(data[0].T), axis=1)
    inset_ax = ax.inset_axes([.6, 0, 0.40, .40])
    inset_ax.scatter(bin_count_sum, elevation_range[:-1], marker='.', s=14)
    inset_ax.set_xlim([0, 160])
    inset_ax.set_xticks([])
    inset_ax.tick_params(axis='y', length=5)
    inset_ax.set_ylabel(ELEVATION_LABEL)
    inset_ax.annotate(label, xy=(138, 3990), fontsize=20)

In [ ]:
fig, (ax1, ax2) = plt.subplots(**HIST2D_PLOT)

h2d_data_sfm = plot_hist2d(
    ax1,
    positive_sfm['sfm_snow_depth'],
    positive_sfm['elevation'],
    'SfM',
    **hist_opts,
)
ax1.set_ylabel(ELEVATION_LABEL)
ax1.annotate('a)', xy=(49.8, 4035), fontsize=20)
# scatter_inset(ax1, h2d_data_sfm, 'c)')

h2d_data_aso = plot_hist2d(
    ax2,
    positive_sfm['aso_snow_depth'],
    positive_sfm['elevation'],
    'ASO',
    **hist_opts,
)
ax2.annotate('b)', xy=(49.8, 4035), fontsize=20)
# scatter_inset(ax2, h2d_data_aso, 'd)')

# fig.suptitle('Snow Depth distribution by elevation')
PlotBase.insert_colorbar(ax2, h2d_data_aso[3], 'Count', **COLOR_BAR_ATTR);

In [ ]:
fig, (ax1, ax2) = plt.subplots(**HIST2D_PLOT)

bin_count_sum = np.count_nonzero(~np.isnan(h2d_data_sfm[0].T), axis=1)
ax1.axhline(y=3550, color='orange', linestyle='--')
ax1.scatter(bin_count_sum, elevation_range[:-1], s=10)
ax1.set_ylabel(ELEVATION_LABEL)
ax1.set_xlabel('Number of bins')

bin_count_sum = np.count_nonzero(~np.isnan(h2d_data_aso[0].T), axis=1)
ax2.axhline(y=3500, color='orange', linestyle='--')
ax2.scatter(bin_count_sum, elevation_range[:-1], s=10)
ax2.set_xlabel('Number of bins')

plt.suptitle('Bin count per elevation band', fontsize=16)
ax1.set_xlim(left=0);

In [ ]:
fig, ax1 = plt.subplots(figsize=(6, 6), dpi=150)

h1 = plot_hist2d(
    ax1,
    positive_sfm['sfm_snow_depth'],
    positive_sfm['aso_snow_depth'],
    '',
    bins=[bins, bins],
    vmin=0,
    vmax=150,
    cmin=1,
)
ax1.add_line(mlines.Line2D([-1, 16], [-1, 16], linestyle='--', color='orange'))
ax1.set_xticks(np.arange(0, 16, 1))
ax1.set_yticks(np.arange(0, 16, 1))
ax1.set_xlim(0, 15)
ax1.set_ylim(0, 15)
ax1.set_ylabel('ASO snow depth (m)')
ax1.set_xlabel('SfM snow depth (m)')
ax1.xaxis.set_ticks_position('bottom');

In [ ]:
vegetation = ((positive_sfm.casi_class == 'Vegetation') | (positive_sfm.casi_class == 'Water'))
open_areas = ~vegetation

In [ ]:
fig, (ax1, ax2) = plt.subplots(**HIST2D_PLOT)

h1 = plot_hist2d(
    ax1,
    positive_sfm[vegetation]['sfm_snow_depth'],
    positive_sfm[vegetation]['elevation'],
    'SfM',
    **hist_opts,
)
ax1.set_ylabel(ELEVATION_LABEL)

h2 = plot_hist2d(
    ax2,
    positive_sfm[vegetation]['aso_snow_depth'],
    positive_sfm[vegetation]['elevation'],
    'ASO',
    **hist_opts,
)

plt.suptitle('Snow Depth in Vegetation')
PlotBase.insert_colorbar(ax2, h2[3], 'count', **COLOR_BAR_ATTR);

In [ ]:
fig, (ax1, ax2) = plt.subplots(**HIST2D_PLOT)

h1 = plot_hist2d(
    ax1,
    positive_sfm[open_areas]['sfm_snow_depth'],
    positive_sfm[open_areas]['elevation'],
    'SfM',
    **hist_opts,
)
ax1.set_ylabel(ELEVATION_LABEL)


h2 = plot_hist2d(
    ax2,
    positive_sfm[open_areas]['aso_snow_depth'],
    positive_sfm[open_areas]['elevation'],
    'ASO',
    **hist_opts,
)
plt.suptitle('Snow Depth in Open Areas')
PlotBase.insert_colorbar(ax2, h2[3], 'count', **COLOR_BAR_ATTR);

# Snow Depth Difference (SfM - ASO)

In [ ]:
cmap = plt.get_cmap("tab20c")
cmap = cmap(np.arange(4)*4, alpha=0.7)

In [ ]:
fig, (ax1) = plt.subplots(
    1, 1, 
    figsize=(8,8),
)
bin_width = 0.1
bins = np.arange(
    math.floor(df.sd_difference.min()) - bin_width, 
    math.ceil(df.sd_difference.max()) + bin_width, 
    bin_width
)
stack = []

for casi_class in CASI_CLASSES:
    stack.append(df[df.casi_class == casi_class].sd_difference)

ax1.hist(
    stack,
    bins=bins,
    label=CASI_CLASSES,
    stacked=True,
    color=cmap,
#     histtype='step',
)
ax1.axvline(0, color='black', linewidth=.4, alpha=1)
ax1.set_xlabel(SNOW_DEPTH_LABEL)
ax1.set_xlim(-5, 5)
ax1.set_ylabel('Count')
ax1.legend(loc='upper left');

In [ ]:
positive_sfm.sd_difference.agg([np.mean, np.median, np.std])

# SfM measurement errors

**Snow Depth binning**
* 0.1 m from -25 to 15 m
* 0.25 m from 15 m to 30 m

In [ ]:
max_bin = max(
    math.fabs(math.floor(negative_sfm.sfm_snow_depth.min())),
    math.ceil(positive_sfm.sfm_snow_depth.max())
)
bins = np.concatenate((
    np.arange(-max_bin, 0, 0.1),
    np.arange(0, 15.10, 0.10),
    np.arange(15, max_bin, 0.25),
))

hist_opts['bins'] = [bins, elevation_range]
HIST2D_PLOT = dict(ncols=2, sharey=True, **FIGURE_SPECS)

In [ ]:
fig, (ax1, ax2) = plt.subplots(**HIST2D_PLOT)

h1 = plot_hist2d(
    ax1,
    negative_sfm['sfm_snow_depth'],
    negative_sfm['elevation'],
    'SfM',
    **hist_opts,
)
ax1.set_ylabel(ELEVATION_LABEL)
ax1.set_xlim(right=0.)
ax1.yaxis.set_ticks_position('both')

h2 = plot_hist2d(
    ax2,
    negative_sfm['aso_snow_depth'],
    negative_sfm['elevation'],
    'ASO',
    **hist_opts,
)
ax2.set_xlim(left=-0.2)

fig.suptitle('Snow Depth measurement errors by Elevation')
PlotBase.insert_colorbar(ax2, h2[3], 'count', **COLOR_BAR_ATTR);

In [ ]:
vegetation = ((negative_sfm.casi_class == 'Vegetation') | (negative_sfm.casi_class == 'Water'))
open_areas = ~vegetation

### Open Areas

In [ ]:
fig, (ax1, ax2) = plt.subplots(**HIST2D_PLOT)

h1 = plot_hist2d(
    ax1,
    negative_sfm[open_areas]['sfm_snow_depth'],
    negative_sfm[open_areas]['elevation'],
    'SfM',
    **hist_opts,
)
ax1.set_ylabel(ELEVATION_LABEL)
ax1.set_xlim(right=0.)

h2 = plot_hist2d(
    ax2,
    negative_sfm[open_areas]['aso_snow_depth'],
    negative_sfm[open_areas]['elevation'],
    'ASO',
    **hist_opts,
)
ax2.set_xlim(left=-0.2)

fig.suptitle('Snow Depth measurement errors by Elevation in open areas')
PlotBase.insert_colorbar(ax2, h2[3], 'count', **COLOR_BAR_ATTR);

### Vegetated areas

In [ ]:
fig, (ax1, ax2) = plt.subplots(**HIST2D_PLOT)

h1 = plot_hist2d(
    ax1,
    negative_sfm[vegetation]['sfm_snow_depth'],
    negative_sfm[vegetation]['elevation'],
    'SfM',
    **hist_opts,
)
ax1.set_ylabel(ELEVATION_LABEL)
ax1.set_xlim(right=0.)

h2 = plot_hist2d(
    ax2,
    negative_sfm[vegetation]['aso_snow_depth'],
    negative_sfm[vegetation]['elevation'],
    'ASO',
    **hist_opts,
)
ax2.set_xlim(left=-0.2)

fig.suptitle('Snow Depth measurement errors by Elevation in vegetated areas')
PlotBase.insert_colorbar(ax2, h2[3], 'count', **COLOR_BAR_ATTR);

### SfM vs ASO depth

In [ ]:
sfm_bins = np.arange(-max_bin, 0, 0.1)
aso_bins = np.concatenate((
    np.arange(0, 2.0, 0.05),
    np.arange(2, 10.1, 0.10)
))
xticks = np.concatenate((
    np.arange(-30, -20, 5),
    np.arange(-20, 0, 2)
))

fig, ax1 = plt.subplots(figsize=(10, 4), dpi=150)
ax1.set_facecolor('whitesmoke')

h1 = ax1.hist2d(
    negative_sfm['sfm_snow_depth'],
    negative_sfm['aso_snow_depth'],
    bins=[sfm_bins, aso_bins],
    vmin=0,
    vmax=150,
    cmin=1,
)
ax1.tick_params(axis='both', length=6)
ax1.yaxis.set_label_position("right")
ax1.yaxis.tick_right()
ax1.set_xlim(-40, 0)
ax1.set_xticks(xticks)
ax1.set_ylim(0, 5)
ax1.set_ylabel('ASO snow depth (m)', rotation=270, labelpad=14)
ax1.set_xlabel('SfM snow depth (m)')
ax1.xaxis.set_ticks_position('bottom')
PlotBase.insert_colorbar(ax1, h1[3], 'Count', right=0.85, rect=COLOR_BAR_ATTR['rect']);

## Negative by CASI classification

In [ ]:
fig, ax1 = plt.subplots(
    figsize=(10.2, 4),
    dpi=150,
)

insert_start = -5

bins = np.concatenate((
    [math.floor(negative_sfm.sfm_snow_depth.min())],
    np.arange(-40, insert_start, 0.1),
    np.arange(insert_start, 0.01, 0.01),
))
stack = []

for casi_class in CASI_CLASSES:
    stack.append(negative_sfm[negative_sfm.casi_class == casi_class].sfm_snow_depth)

ax1.hist(
    stack,
    bins=bins,
    label=CASI_CLASSES,
    stacked=True,
    color=cmap,
#     histtype='step',
)
ax1.set_xlabel(SNOW_DEPTH_LABEL)
ax1.set_xlim(insert_start, bins.max())
PlotBase.format_axes_scientific(ax1, 'y', (4, 4))
PlotBase.move_yaxis_label_right(ax1)

ax2 = ax1.inset_axes([0.02, 0.4, 0.6, .6])
ax2.hist(
    stack,
    bins=bins,
    label=CASI_CLASSES,
    stacked=True,
    color=cmap,
)
ax2.set_xlim(-30, insert_start)
ax2.set_ylim(top=3000)
ax2.set_yticks(np.arange(0, 3000, 500))
PlotBase.format_axes_scientific(ax2, 'y', (3, 3))
PlotBase.move_yaxis_label_right(ax2)
ax2.legend(loc='upper left', fontsize=11);

## SfM negative with aspect

In [ ]:
bin_min = math.floor(negative_sfm.sfm_snow_depth.min())
bins = np.arange(0, bin_min - 0.1, -0.2)
bins = np.flip(bins)

aspect_range = np.arange(0, 361, 1)

hist_opts = dict(
    bins=[aspect_range, bins],
    vmin=0,
    vmax=200,
    cmin=1,
)

In [ ]:
fig = plt.figure(figsize=(15,8))
ax = fig.gca()
ax.set_facecolor('whitesmoke')
ha = ax.hist2d(
    negative_sfm['aspect'],
    negative_sfm['sfm_snow_depth'],
    **hist_opts,
)
PlotBase.insert_colorbar(ax, ha[3], 'Count')
ax.set_ylabel(SNOW_DEPTH_LABEL)
ax.set_xlabel('Aspect');

In [ ]:
aspect_stats = negative_sfm[['sfm_snow_depth', 'aspect']].groupby('aspect')

aspect_count = aspect_stats.count()
aspect_medians = aspect_stats.median()
aspect_means = aspect_stats.mean()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(14, 24))
ax1.scatter(aspect_count.index, aspect_count.sfm_snow_depth)
ax1.set_title('Aspect count')

ax2.scatter(aspect_medians.index, aspect_medians.sfm_snow_depth)
ax2.set_ylabel('Median SD difference')
ax2.set_title('Median difference by aspect');

ax3.scatter(aspect_means.index, aspect_means.sfm_snow_depth)
ax3.set_xlim(aspect_range.min(), aspect_range.max())
ax3.set_ylabel('Mean SD difference')
ax3.set_xlabel('Aspect in degree')
ax3.set_title('Mean difference by aspect');

In [ ]:
aspect_count.sort_values('sfm_snow_depth', ascending=False).head(10).sort_index()

## Vegetation and Water category masked out

In [ ]:
vegetation_free = negative_sfm.query('casi_class != "Vegetation" and casi_class != "Water"')
vegetation_free_aspects = vegetation_free.groupby('aspect')
vegetation_free_aspects_median = vegetation_free_aspects.median()
vegetation_free_aspects_count = vegetation_free_aspects.count()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(15,24))
ax1.set_facecolor('whitesmoke')
ha = ax1.hist2d(
    vegetation_free['aspect'],
    vegetation_free['sfm_snow_depth'],
    **hist_opts,
)
ax1.set_ylabel(SNOW_DEPTH_LABEL)
ax1.set_title('Negative values with aspect - No Vegetation')

ax2.scatter(vegetation_free_aspects_median.index, vegetation_free_aspects_median.sfm_snow_depth)
ax2.set_title('Median difference by aspect - No Vegetation')
ax2.axvline(HILLSHADE_SNOW_ON['azimuth'], color='goldenrod', linestyle='--')
ax2.annotate('Snow On\n Sun Angle', xy=(HILLSHADE_SNOW_ON['azimuth'] + 1, -0.15))
ax2.axvline(HILLSHADE_SNOW_ON['azimuth'] + 180, color='dimgrey', linestyle='--')
ax2.annotate('Snow On\n Shade', xy=(HILLSHADE_SNOW_ON['azimuth'] + 181, -0.5))
ax2.axvline(HILLSHADE_SNOW_FREE['azimuth'], color='goldenrod', linestyle='--')
ax2.annotate('Snow Free\n Sun Angle', xy=(HILLSHADE_SNOW_FREE['azimuth'] + 1, -0.5))
ax2.axvline(HILLSHADE_SNOW_FREE['azimuth'] % 180, color='dimgrey', linestyle='--')
ax2.annotate('Snow Free\n Shade', xy=((HILLSHADE_SNOW_FREE['azimuth'] + 181) % 180, -0.15))
ax2.set_ylabel('Median SD difference')

ax3.scatter(vegetation_free_aspects_count.index, vegetation_free_aspects_count.sfm_snow_depth)
ax3.set_title('Count of aspects - No Vegetation');
ax3.set_ylabel('Median SD difference')

ax3.set_xlim(aspect_range.min(), aspect_range.max())
ax3.set_xlabel('Aspect in degree');

# PlotBase.insert_colorbar(ax1, ha[3], 'count');

## SfM values with slope

In [ ]:
slope_range = np.arange(0, 91, 1)

hist_opts['bins'] = [slope_range, bins]

In [ ]:
fig = plt.figure(figsize=(15,10))
ax = fig.gca()
ax.set_facecolor('whitesmoke')
hs = ax.hist2d(
    negative_sfm['slope'],
    negative_sfm['sfm_snow_depth'],
    **hist_opts
)
PlotBase.insert_colorbar(ax, hs[3], 'count')
ax.set_ylabel(SNOW_DEPTH_LABEL)
ax.set_xlabel('Slope')
ax.set_title('Negative values with slope');

In [ ]:
slope_stats = negative_sfm[['sfm_snow_depth', 'slope']].groupby('slope')
slope_count = slope_stats.count()
slope_median = slope_stats.median()
slope_means = slope_stats.mean()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(15, 24))

ax1.scatter(slope_count.index, slope_count.sfm_snow_depth)
ax1.set_title('Slope Count')

ax2.scatter(slope_median.index, slope_median.sfm_snow_depth)
ax2.set_ylabel('Median SD difference')
ax2.set_title('Median difference by slope')

ax3.scatter(slope_means.index, slope_means.sfm_snow_depth)
ax3.set_xlim(slope_range.min(), slope_range.max())
ax3.set_xlabel('Slope in degree')
ax3.set_ylabel('Mean SD difference')
ax3.set_title('Mean difference by slope');

In [ ]:
vegetation_free_slopes = vegetation_free.groupby('slope')
vegetation_free_slopes_median = vegetation_free_slopes.median()
vegetation_free_slopes_mean = vegetation_free_slopes.mean()
vegetation_free_slopes_count = vegetation_free_slopes.count()

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, sharex=True, figsize=(15,32))

ax1.set_facecolor('whitesmoke')
hs = ax1.hist2d(
    vegetation_free['slope'],
    vegetation_free['sfm_snow_depth'],
    **hist_opts
)
PlotBase.insert_colorbar(ax1, hs[3], 'count')
ax1.set_ylabel(SNOW_DEPTH_LABEL)
ax1.set_title('Negative values with slope - No Vegetation')

ax2.scatter(vegetation_free_slopes_median.index, vegetation_free_slopes_median.sfm_snow_depth)
ax2.axhline(-1, color='orange', linestyle='--', alpha=0.8)
ax2.axhline(-4, color='orange', linestyle='--', alpha=0.8)
ax2.axvline(51, color='orange', linestyle='--', alpha=0.8)
ax2.axvline(75, color='orange', linestyle='--', alpha=0.8)
ax2.set_ylabel('Median SD difference')
ax2.set_title('Median difference by slope - No vegetation')

ax3.scatter(vegetation_free_slopes_mean.index, vegetation_free_slopes_mean.sfm_snow_depth)
ax3.axvline(51, color='orange', linestyle='--', alpha=0.8)
ax3.axvline(75, color='orange', linestyle='--', alpha=0.8)
ax3.set_ylabel('Mean SD difference')
ax3.set_title('Mean difference by slope - No vegetation')

ax4.scatter(vegetation_free_slopes_count.index, vegetation_free_slopes_count.sfm_snow_depth)
ax4.axvline(51, color='orange', linestyle='--', alpha=0.8)
ax4.axvline(75, color='orange', linestyle='--', alpha=0.8)
ax4.set_title('Count of slope angels - No vegetation')
ax4.set_ylim(bottom=0)

ax4.set_xlabel('Slope in degree')
ax4.set_xlim(slope_range.min(), slope_range.max());

## Hillshade comparison (No Vegetation)

## Snow free SfM

Azimuth: 247

Altitude: 32

In [ ]:
HILLSHADE_MIN = 0
HILLSHADE_MAX = 255

hillshade_range = np.arange(HILLSHADE_MIN, HILLSHADE_MAX + 1, 1)

hist_opts['bins'] = [hillshade_range, bins]

In [ ]:
hillshade_snow_free = vegetation_free.groupby('hillshade_snow_free')
hillshade_snow_free_median = hillshade_snow_free.median()
hillshade_snow_free_count = hillshade_snow_free.count()

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(15,24))

ax1.set_facecolor('whitesmoke')
ax1.set_facecolor('whitesmoke')
ha = ax1.hist2d(
    negative_sfm.hillshade_snow_free,
    negative_sfm.sfm_snow_depth,
    **hist_opts,
)
PlotBase.insert_colorbar(ax1, ha[3], 'count')
ax1.set_ylabel(SNOW_DEPTH_LABEL)
ax1.set_title('Negative values with Snow Free hillshade')

ax2.scatter(hillshade_snow_free_median.index, hillshade_snow_free_median.sfm_snow_depth)
ax2.set_ylabel('Median SD difference')
ax2.set_title('Median difference')

ax3.scatter(hillshade_snow_free_count.index, hillshade_snow_free_count.sfm_snow_depth)
ax3.set_ylabel('Count')
ax3.set_title('Count by hillshade')

ax3.set_xlim(HILLSHADE_MIN, HILLSHADE_MAX)
ax3.set_xlabel('Hillshade');

## SfM snow on

Azimuth: 100

Altitude: 47

In [ ]:
hillshade_snow_on = vegetation_free.groupby('hillshade_snow_on')
hillshade_snow_on_median = hillshade_snow_on.median()
hillshade_snow_on_count = hillshade_snow_on.count()

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(15,24))

ax1.set_facecolor('whitesmoke')
ax1.set_facecolor('whitesmoke')
ha = ax1.hist2d(
    negative_sfm.hillshade_snow_on,
    negative_sfm.sfm_snow_depth,
    **hist_opts,
)
PlotBase.insert_colorbar(ax1, ha[3], 'count')
ax1.set_ylabel(SNOW_DEPTH_LABEL)
ax1.set_title('Negative values with Snow On hillshade')

ax2.scatter(hillshade_snow_on_median.index, hillshade_snow_on_median.sfm_snow_depth)
ax2.set_ylabel('Median SD difference')
ax2.set_title('Median difference')

ax3.scatter(hillshade_snow_on_count.index, hillshade_snow_on_count.sfm_snow_depth)
ax3.set_ylabel('Count')
ax3.set_title('Count by hillshade')

ax3.set_xlim(HILLSHADE_MIN, HILLSHADE_MAX)
ax3.set_xlabel('Hillshade');